# Start-up

In [14]:
import sys
import time
from numpy import *
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from control_hugo import red_pitaya_control,red_pitaya_app
import socket
import struct

AppName = "lock_in+pid_for_qudi"
rp_ip = 'rp-f0a83e.local'
port = 22
filename = "test.npz"
pwd = 'quantuM2015'

rp=red_pitaya_app(AppName=AppName,host=rp_ip,port=port,filename=filename,password=pwd)

auth method: pass
Remote Command: uname -a
Remote Command: echo $SSH_CONNECTION
rp.osc.load(): /opt/redpitaya/www/apps/lock_in+pid_for_qudi/py/osc.py
Remote Command: /opt/redpitaya/www/apps/lock_in+pid_for_qudi/py/osc.py
rp.lock_in+pid_for_qudi.load(): /opt/redpitaya/www/apps/lock_in+pid_for_qudi/py/lock.py
Remote Command: /opt/redpitaya/www/apps/lock_in+pid_for_qudi/py/lock.py
Remote Command: cat /opt/redpitaya/www/apps/lock_in+pid_for_qudi/index.html
Remote Command: /opt/redpitaya/bin/calib -r -v


In [ ]:
rp.ssh.close()

In [ ]:
print(rp.check_connection())

# Example

In [ ]:
import logging
logging.basicConfig()
logging.getLogger("paramiko").setLevel(logging.WARNING)
rp.verbose = False

In [ ]:
# Set params
rp.lock.ramp_step = 1000
rp.lock.ramp_low_lim = 0
rp.lock.ramp_hig_lim = 8191
rp.lock.ramp_enable = 1
rp.lock.out2_sw = 12

rp.lock.oscA_sw = 1
rp.lock.oscB_sw = 2
rp.lock.trig_sw = 8

rp.get_adc_dac_calib()

trigger_type = 6  # 6 is external, 1 is manual

In [ ]:
# Decimation only allows this values: 1,8,64, 1024, 8192, 65536
# The oscilloscope data points will be separated by 2^(dec-1) * 8 ns
dec = 1 # [1,8,64, 1024, 8192, 65536]

rp.osc_trig_fire(trig=trigger_type,dec=dec)
sleep(dec *8e-9*2**14 + 0.2)
rp.get_curv(log='ruido info' )

rp.save()

In [ ]:
# Access last acquisition values
ch1_val = mean( rp.data[-1][2]['ch1'])
ch1_err =  std( rp.data[-1][2]['ch1'])

ch2_val = mean( rp.data[-1][2]['ch2'])
ch2_err =  std( rp.data[-1][2]['ch2'])

ch1_act = (ch1_val + rp.calib_params['FE_CH1_DC_offs'])*float(rp.calib_params['FE_CH1_FS_G_HI'])/2**32*100/8192
ch2_act = (ch2_val + rp.calib_params['FE_CH2_DC_offs'])*float(rp.calib_params['FE_CH2_FS_G_HI'])/2**32*100/8192


In [ ]:
# plot last acquisition
rp.plot(num=0)

In [ ]:
rp.get_adc_dac_calib()

print(repr(rp.calib_params))

In [ ]:
rp.plot(num=1)

In [ ]:
rp.plot(num=-1)

# Test Real-time acquisition

## Single acquisitions

In [ ]:
rp.get_curv()

In [ ]:
len(rp.data)

In [ ]:
latest_acq = rp.data[-1]
latest_acq_number = latest_acq[0]
latest_acq_time = latest_acq[1]
latest_acq_data = latest_acq[2]
print("Acq_num: ",latest_acq_number,"  | Timestamp: ",datetime.fromtimestamp(latest_acq_time))

In [ ]:
t_idx = latest_acq_data['i']
print("Time data: ",t_idx)

In [ ]:
ch1 = latest_acq_data['ch1']
print("Ch. 1 data: ",ch1)

In [ ]:
ch2 = latest_acq_data['ch2']
print("Ch. 2 data: ",ch2)

In [ ]:
dec = latest_acq_data['dec']
tau = 8*2**latest_acq_data['dec']
t = [x * tau for x in t_idx]
print(t)

In [ ]:
print("dec:",dec,"=> Tau: ",tau,"ns")

In [ ]:
plt.plot(t,ch1)
plt.plot(t,ch2)

## Stream

In [7]:
# Start streaming specific signals
rp.start_streaming(signals='in1 in2')

# Collect data for a while (e.g., 10 seconds)
# time.sleep(10)

# Stop streaming when done
# rp.stop_streaming()

Getting streaming for [in1 in2]
start_streaming(): Getting streaming for [in1 in2]
start_streaming():
rp_ip: 134.60.243.125
local_ip:  134.60.243.151 

cmd:  /opt/redpitaya/www/apps/lock_in+pid_for_qudi/py/data_dump.py  -s 134.60.243.151 -p 6000 --params in1 in2 

start_streaming(): filename=20250204_151948_dump.bin
remote: /opt/redpitaya/www/apps/lock_in+pid_for_qudi/py/data_dump.py  -s 134.60.243.151 -p 6000 --params in1 in2


In [15]:
rp.stop_streaming()

Streaming never started


In [27]:
HOST = "0.0.0.0"  # Listen on all available interfaces
PORT = 6000
FORMAT = "!fll"  # Struct format: float, long, long
SIZE = struct.calcsize(FORMAT)  # Calculate expected bytes size
print(SIZE)

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind(('', PORT))
    s.listen(1)  # Allow one connection
    print(f"Listening on port {PORT}...")

    connection, client_address = s.accept()
    print(f"Connection from {client_address}")

    start_time = time.time()
    while time.time() - start_time < 10:  # Run for 10 seconds
        try:
            # s.settimeout(1)  # Timeout for receiving
            data = connection.recv(SIZE)
            unpacked_data = struct.unpack(FORMAT, data)
            print(f"Received from {client_address}: {unpacked_data}")
        except socket.timeout:
            continue
    print("Timeout reached. Exiting.")

12
Listening on port 6000...
Connection from ('134.60.243.125', 58250)
Received from ('134.60.243.125', 58250): (0.0055310726165771484, 0, 0)
Received from ('134.60.243.125', 58250): (0.008681297302246094, 0, 0)
Received from ('134.60.243.125', 58250): (0.011844873428344727, 0, 0)
Received from ('134.60.243.125', 58250): (0.014896154403686523, 0, 0)
Received from ('134.60.243.125', 58250): (0.017935514450073242, 0, 0)
Received from ('134.60.243.125', 58250): (0.020976543426513672, 0, 0)
Received from ('134.60.243.125', 58250): (0.024013280868530273, 0, 0)
Received from ('134.60.243.125', 58250): (0.02705240249633789, 0, 0)
Received from ('134.60.243.125', 58250): (0.03016376495361328, 0, 0)
Received from ('134.60.243.125', 58250): (0.03319907188415527, 0, 0)
Received from ('134.60.243.125', 58250): (0.03623318672180176, 0, 0)
Received from ('134.60.243.125', 58250): (0.03934168815612793, 0, 0)
Received from ('134.60.243.125', 58250): (0.042388200759887695, 0, 0)
Received from ('134.60.